<a href="https://colab.research.google.com/github/Scyzentraz/modulajar/blob/main/professionalgradenibos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Langkah 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Langkah2. Install Model dan Dependensi yang dibutuhkan

#Install Real esrgan
import os

if not os.path.exists("Real-ESRGAN"):
    !git clone https://github.com/xinntao/Real-ESRGAN.git

#Install dependensi
%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop
!pip install tqdm
!pip install pillow

# Install waifu2x-caffe headless (pakai waifu2x-ncnn-vulkan yang ringan dan cepat)
!wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20220728/waifu2x-ncnn-vulkan-20220728-ubuntu.zip
!unzip waifu2x-ncnn-vulkan-20220728-ubuntu.zip -d waifu2x
!chmod +x waifu2x/waifu2x-ncnn-vulkan

In [ ]:
#Langkah 3. Patch bug rgb to grayscale (opsional)
file_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"

with open(file_path, 'r') as file:
    file_data = file.readlines()

# Hanya patch jika belum dipatch
patched = False
for i, line in enumerate(file_data):
    if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
        file_data[i] = 'from torchvision.transforms.functional import rgb_to_grayscale\n'
        patched = True
        break

if patched:
    with open(file_path, 'w') as file:
        file.writelines(file_data)
    print("✅ Bug torchvision berhasil dipatch.")
else:
    print("✅ Tidak perlu patch, sudah sesuai.")

In [ ]:
#Langkah 4.1 siapkan file input (manual di google drive, masukan gambar awal ke folder /upscale/input)
#Langkah 4.2 konfigurasi batch
import os
from glob import glob




# Konfigurasi
source = "image"
input_dir = "/content/drive/MyDrive/upscale/input"
output_dir_waifu2x = "/content/drive/MyDrive/upscale/waifu2x"
output_dir_1 = "/content/drive/MyDrive/upscale/output1"
output_dir_2 = "/content/drive/MyDrive/upscale/output2"
output_dir_3 = "/content/drive/MyDrive/upscale/output3"
final_output_dir = "/content/drive/MyDrive/upscale/final_jpeg"
model = "RealESRGAN_x2plus"
scale = 2
face_enhance = False


# Buat folder output jika belum ada
os.makedirs(output_dir_waifu2x, exist_ok=True)
os.makedirs(output_dir_1, exist_ok=True)
os.makedirs(output_dir_2, exist_ok=True)
os.makedirs(output_dir_3, exist_ok=True)
os.makedirs(final_output_dir, exist_ok=True)

# Ambil semua file jpg/jpeg
image_paths = sorted(
    glob(os.path.join(input_dir, "*.jpg")) +
    glob(os.path.join(input_dir, "*.jpeg")) +
    glob(os.path.join(input_dir, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")

In [ ]:
#Langkah 5.0, preprocess waifu2x

import subprocess
os.makedirs(output_dir_waifu2x, exist_ok=True)


for img in tqdm(image_paths, desc="🔇 Denoising Waifu2x"):
    filename = os.path.basename(img)
    output_path = os.path.join(output_dir_waifu2x, os.path.splitext(filename)[0] + ".png")
    command = f"./waifu2x/waifu2x-ncnn-vulkan -i \"{img}\" -o \"{output_path}\" -n 1 -s 1"
    subprocess.run(command, shell=True)

In [ ]:

# Langkah 5.1 Ambil semua file jpg/jpeg/png, gunakan output dari waifu2x sebagai input ke tahap upscale pertama
image_paths = sorted(
    glob(os.path.join(output_dir_waifu2x, "*.jpg")) +
    glob(os.path.join(output_dir_waifu2x, "*.jpeg")) +
    glob(os.path.join(output_dir_waifu2x, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_waifu2x):
    print("❌ Tidak ada output dari tahap preprocess!")

In [ ]:
#Langkah 5.1.1 Upscale tahap pertama
from tqdm import tqdm

code = "inference_realesrgan.py"
tile = 0  # ukuran tile, bisa disesuaikan jika masih kehabisan memori

# Jalankan batch dengan progress bar

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.splitext(os.path.basename(input_path))[0] + ".png"
    output_path = os.path.join(output_dir_1, filename)
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale} --tile {tile}'

    if face_enhance:
        command += " --face_enhance"

    os.system(command)

In [ ]:
# Langkah 5.2 Ambil semua file jpg/jpeg/png, gunakan output dari tahap pertama sebagai input ke tahap upscale kedua

image_paths = sorted(
    glob(os.path.join(output_dir_1, "*.jpg")) +
    glob(os.path.join(output_dir_1, "*.jpeg")) +
    glob(os.path.join(output_dir_1, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_1):
    print("❌ Tidak ada output dari tahap pertama!")

In [ ]:
#Langkah 5.2.1 upscale tahap 2
from tqdm import tqdm
import os

code = "inference_realesrgan.py"
tile = 400  # ukuran tile, bisa disesuaikan kalau masih kehabisan memori

# Jalankan batch dengan progress bar

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.splitext(os.path.basename(input_path))[0] + ".png"
    output_path = os.path.join(output_dir_2, filename)
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale} --tile {tile}'


    if face_enhance:
        command += " --face_enhance"

    print("\n[📤 Menjalankan command:]")
    print(command)  # log command yang dijalankan

    exit_code = os.system(command)

    if exit_code != 0:
        print(f"[❌ Gagal memproses {input_path}] (exit code: {exit_code})")
    else:
        print(f"[✅ Berhasil memproses {input_path}]")

In [ ]:
#Langkah 5.3 Ambil semua file jpg/jpeg/png, gunakan output dari tahap kedua sebagai input ke tahap upscale ketiga

image_paths = sorted(
    glob(os.path.join(output_dir_2, "*.jpg")) +
    glob(os.path.join(output_dir_2, "*.jpeg")) +
    glob(os.path.join(output_dir_2, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_2):
    print("❌ Tidak ada output dari tahap kedua!")

In [ ]:
#Langkah 5.3.1 upscale tahap 3
from tqdm import tqdm
import subprocess
import os

tile = 0  #
tile_pad = 10
code = "inference_realesrgan.py"

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.splitext(os.path.basename(input_path))[0] + ".png"
    output_path = os.path.join(output_dir_3, filename)
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale} --tile {tile} --tile_pad {tile_pad}'

    if face_enhance:
        command += " --face_enhance"

    print("[📤 Menjalankan command:]")
    print(command)

    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"[❌ Gagal memproses {input_path}]")
        print("STDERR:", result.stderr)
    else:
        print(f"[✅ Berhasil memproses {input_path}]")

In [ ]:
#Langkah 6. Ambil semua file jpg/jpeg/png, gunakan output dari tahap ketiga sebagai input ke tahap jpeg 100%

image_paths = sorted(
    glob(os.path.join(output_dir_3, "*.jpg")) +
    glob(os.path.join(output_dir_3, "*.jpeg")) +
    glob(os.path.join(output_dir_3, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")
if not os.listdir(output_dir_3):
    print("❌ Tidak ada output dari tahap ketiga!")

In [ ]:
# Langkah 6.1: Konversi output akhir ke JPEG kualitas 100%

from PIL import Image
from glob import glob
import os


os.makedirs(final_output_dir, exist_ok=True)

for img_path in tqdm(image_paths, desc="🧼 Konversi ke JPEG 100%"):
    with Image.open(img_path) as img:
        img = img.convert("RGB")
        filename = os.path.splitext(os.path.basename(img_path))[0] + ".jpg"
        out_path = os.path.join(final_output_dir, filename)
        img.save(out_path, format="JPEG", quality=100, subsampling=0)

print("✅ Semua gambar disimpan ulang sebagai JPEG kualitas 100% di:", final_output_dir)

In [ ]:
#Langkah 7: Pengecekan dan verifikasi
import os

#cek ukuran dan format
total_size = sum(os.path.getsize(f) for f in glob(os.path.join(final_output_dir, "*.jpg")))
print(f"📦 Total size semua JPEG 100%: {total_size / (1024 * 1024):.2f} MB")

#validasi format file
for path in glob(os.path.join(final_output_dir, "*.jpg")):
    try:
        with Image.open(path) as img:
            img.verify()
    except Exception as e:
        print(f"❌ Corrupt file: {path} - {e}")